### Imports

In [3]:
import pandas as pd

### Input, Manipulation, and Output

In [24]:
folder_name = "BULK-StrictEqualityCatRestriction3/" # This must end in a slash
output_name = "All Restaurant Food Count by Age & Restaurant"
outname = "./ModelOutput/" + folder_name + output_name + ".xlsx"
multi_obj = False

ages = [2, 4, 9, 14, 19, 31, 51]
genders = ["Male", "Female"]
objectives = []
foods = []
for age in ages:
    for gender in genders:
        in_name = f"{gender}-{age}"
        inname = "./ModelOutput/" + folder_name + in_name + ".xlsx" 
        ob = pd.read_excel(inname, sheet_name="Rankings")
        ob.loc[:, "Age"] = age
        ob.loc[:, "Gender"] = gender
        objectives.append(ob)
        
        fd = pd.read_excel(inname, sheet_name="Foods")
        fd.loc[:, "Age"] = age
        fd.loc[:, "Gender"] = gender
        foods.append(fd)
ob_bunch = pd.concat(objectives).rename({"Unnamed: 0": "Restaurant"}, axis=1)
fd_bunch = pd.concat(foods).rename({"Unnamed: 0": "Restaurant"}, axis=1)


with pd.ExcelWriter(outname) as writer:
    for res in pd.unique(fd_bunch["Restaurant"]):
        print(res, end=", ")
        filtered_foods = fd_bunch[fd_bunch["Restaurant"] == res].loc[:, ["Food", "Objective", "Amount", "Gender", "Age"]]
        ptable = pd.pivot_table(filtered_foods, index=["Food", "Objective"], columns=["Gender", "Age"], observed=True)
        ptable.to_excel(writer, sheet_name=res)
print("Finished!")

Arby's, Baskin-Robbins, Blimpie, Bojangles, Boston Market, Burger King, Chipotle, Culvers, Dairy Queen, Del Taco, Dunkin Donuts, Five Guys, Godfather's Pizza, Jersey Mike's, KFC, Long John Silver's, Panda Express, Popeyes, Raising Cane's, Red Lobster, Sonic, Subway, Taco Bell, Wendy's, Whataburger, Zaxby's, Chick-fil-A, McDonald's, Olive Garden, Finished!
